In [1]:
import numpy as np
import imutils
import glob
import cv2
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
FMS_table =  [(20,22),(285,23),(546,128),(738,120),(926,120),
              (1116,118),(1306,127),(1518,21),(1781,24),(1781,453),
              (1517,453),(1211,380),(926,381),(641,381),(288,451),
              (25,452),(24,236),(286,234),(1518,235),(1781,235)]

FMX_table = [(20,22),(349,21),(630,19),(630,19),(909,18),
             (1189,25),(1189,25),(1445,28),(1771,23),(1775,480),
             (1442,480),(1189,404),(915,403),(633,404),(355,480),
             (20,477),(25,252),(348,248),(1447,250),(1771,249)]

In [3]:
def choice_class(table, pt, Is_FMX):
    w, h = pt[0], pt[1]
    label = -1
    for idx, position in enumerate(FMS_table):
        if position[0]-50 <= w <= position[0]+50 and position[1]-50 <= h <= position[1]+50:
            label = idx+1
            break
            
    if Is_FMX:
        if label == 3:
            label = 3.5
        elif label == 6:
            label = 6.5
        
    return label

In [4]:
%%time
path = "D:\\Lab\\PBL\\Data"
out_label = pd.DataFrame(columns=['folder','image','label'])
out_label_idx = 0
for dirname in tqdm(os.listdir(path)):
    Is_FMX = False
    print('dirname =',dirname)
    templete_image = [i for i in glob.iglob(path + "\\%s/*/*.PNG" % dirname)]
    target_image =  [i for i in glob.iglob(path + "\\%s/*/*FMS*.png" % dirname)]
    if len(target_image)==0:
        Is_FMX = True
        target_image =  [i for i in glob.iglob(path + "\\%s/*/*FMX*.png" % dirname)]
        if len(target_image) == 0:
            print('error dir =',dirname)
            continue
    
    if Is_FMX == True:
        table = FMX_table
    else:
        table = FMS_table
        
    image_o = cv2.imread(target_image[0]) # image
    image = cv2.cvtColor(image_o, cv2.COLOR_BGR2GRAY)

    green = 0

    for index, tmp_img in enumerate(templete_image):
        
        if 'FMS' in tmp_img or 'FMX' in tmp_img:
            continue
        
        template = cv2.imread(tmp_img) # template image

        template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

        loc = False
        threshold = 0.8
        w, h = template.shape[::-1]

        for scale in np.linspace(0.1, 0.9, 80)[::-1]:
        #for scale in np.linspace(0.15, 0.8, 65)[::-1]:
            try:
                resized = imutils.resize(template, width = int(template.shape[1] * scale))
                w, h = resized.shape[::-1]
                res = cv2.matchTemplate(image,resized,cv2.TM_CCOEFF_NORMED)

                loc = np.where( res >= threshold)
                if len( list(zip(*loc[::-1])) ) > 0:
                    break
            except:
                pass
        

        if loc and len( list(zip(*loc[::-1])) ) > 0:
            
            if w < 100 or h < 100:
                print(tmp_img[-10:-4])
                print('w =',w)
                print('h =',h)
                out_label.loc[out_label_idx, 'folder'] = tmp_img.split('\\')[5]
                out_label.loc[out_label_idx, 'image'] = tmp_img.split('\\')[-1][:-4]
                out_label.loc[out_label_idx, 'label'] = -1
                out_label_idx += 1
                continue
                
            for pt in zip(*loc[::-1]):
                cv2.rectangle(image_o, pt, (pt[0] + w, pt[1] + h), (0, green, 255), 2)

#                print(index + 1)
#                print(pt)
#                print(tmp_img)
                #text = str(index + 1)
                text = tmp_img[-10:-4]
                cv2.putText(image_o, text, (pt[0] + w - 100, pt[1] + h - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, green, 255), 1, cv2.LINE_AA)
                
                label = choice_class(table, pt, Is_FMX)
                
                break
        else:
            label = -1
            
        green += 12
        
        out_label.loc[out_label_idx, 'folder'] = tmp_img.split('\\')[5]
        out_label.loc[out_label_idx, 'image'] = tmp_img.split('\\')[-1][:-4]
        out_label.loc[out_label_idx, 'label'] = label
        out_label_idx += 1
        

#     cv2.namedWindow('Matched Template', cv2.WINDOW_NORMAL)
#     cv2.imshow('Matched Template', image_o)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    cv2.imwrite( "truth_template_matching/"+dirname+'.png', image_o );

C:\Users\lab118\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


dirname = 000408
dirname = 000411
dirname = 001742
dirname = 002456
dirname = 002555
dirname = 003262
dirname = 003615
dirname = 003670
dirname = 004151
dirname = 004359
dirname = 004499
2B1F6C
w = 82
h = 131
dirname = 005627
dirname = 007274
dirname = 007501
dirname = 008908
dirname = 010801
dirname = 0109363
dirname = 010953
dirname = 014185
dirname = 014735
dirname = 014783
dirname = 015441
dirname = 016057
dirname = 016165
dirname = 016649
dirname = 017070
dirname = 017374
dirname = 017635
dirname = 018526
dirname = 019462
dirname = 019660
dirname = 019747_1
dirname = 019747_2
dirname = 021033
dirname = 021271
dirname = 022198_1
dirname = 022198_2
dirname = 022702
dirname = 024090
dirname = 025179_1
dirname = 025179_2
dirname = 025271
dirname = 025575
dirname = 026071
dirname = 026110_1
dirname = 026110_2
dirname = 026118_1
dirname = 026118_2
dirname = 026127
dirname = 026545
dirname = 026550
dirname = 026610
dirname = 027035
dirname = 027098
dirname = 028492
dirname = 029560
dirna

In [5]:
out_label = out_label.set_index(keys = ['folder','image'])
out_label.sort_values(['folder','label'],inplace=True)
out_label.to_csv('class_label.csv')
out_label

label
folder           image                        
000408 102419 x  NN_191024_151631_BE78BA     1
                 NN_191024_151636_BE78B6     2
                 NN_191024_151652_BE78B0     3
                 NN_191024_151623_BE78A8     4
                 NN_191024_151736_BE78AA     5
...                                        ...
S613744 021815 x NN_150218_102751_C0A04B    16
                 NN_150218_102701_C0A00C    17
                 NN_150218_105121_C0A022    18
                 NN_150218_105120_C0A01E    19
                 NN_150218_102718_C0A04F    20

[6812 rows x 1 columns]

In [6]:
# %%time
# path = "D:\\Lab\\PBL\\Data"
# dirname = "035015_1"
# templete_image = [i for i in glob.iglob(path + "\\%s/*/*.PNG" % dirname)]
# # target_image =  [i for i in glob.iglob("D:/Lab/PBL/Data/%s/NN*.png" % dirname)]
# target_image =  [i for i in glob.iglob(path + "\\%s/*/*FMS*.png" % dirname)]
# if len(target_image)==0:
#     target_image =  [i for i in glob.iglob(path + "\\%s/*/*FMX*.png" % dirname)]
#     if len(target_image) == 0:
#         print('error dir =',dirname)
# image_o = cv2.imread(target_image[0]) # image
# image = cv2.cvtColor(image_o, cv2.COLOR_BGR2GRAY)

# green = 0

# for index, tmp_img in enumerate(templete_image):
#     template = cv2.imread(tmp_img) # template image

#     template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

#     loc = False
#     threshold = 0.8
#     w, h = template.shape[::-1]

#     for scale in np.linspace(0.1, 0.9, 80)[::-1]:
#         try:
#             resized = imutils.resize(template, width = int(template.shape[1] * scale))
#             w, h = resized.shape[::-1]
#             res = cv2.matchTemplate(image,resized,cv2.TM_CCOEFF_NORMED)

#             loc = np.where( res >= threshold)
#             if len( list(zip(*loc[::-1])) ) > 0:
#                 break
#         except:
#             print(scale)
#             pass

#     if loc and len( list(zip(*loc[::-1])) ) > 0:
#             for pt in zip(*loc[::-1]):
#                     cv2.rectangle(image_o, pt, (pt[0] + w, pt[1] + h), (0, green, 255), 2)

#                     print(index + 1)
#                     print(pt)
#                     print(tmp_img)
#                     text = str(index + 1)

#                     cv2.putText(image_o, text, (pt[0] + w - 100, pt[1] + h - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, green, 255), 1, cv2.LINE_AA)
#                     break
#         #green += 12

# cv2.namedWindow('Matched Template', cv2.WINDOW_NORMAL)
# cv2.imshow('Matched Template', image_o)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.imwrite( "Mapping Results.jpg", image_o );

In [7]:
# templete_image

In [8]:
# tmp_img.split('\\')[5]